In [25]:
import random
import math

# Classe do Agente
class Agent:
    def __init__(self, x, y, hunger, vision=5):
        self.x = x
        self.y = y
        self.hunger = hunger  # Fome inicial
        self.vision = vision  # Raio de visão
        self.alive = True

    def move(self, grid_size):
        """Move o agente aleatoriamente dentro do espaço"""
        if not self.alive:
            return
        dx, dy = random.choice([(0, 1), (1, 0), (0, -1), (-1, 0)])
        self.x = max(0, min(self.x + dx, grid_size - 1))
        self.y = max(0, min(self.y + dy, grid_size - 1))
        self.hunger += 1  # Fome aumenta a cada movimento

    def move_towards(self, target_x, target_y):
        """Move o agente em direção à comida"""
        if not self.alive:
            return
        if self.x < target_x:
            self.x += 1
        elif self.x > target_x:
            self.x -= 1
        if self.y < target_y:
            self.y += 1
        elif self.y > target_y:
            self.y -= 1
        self.hunger += 1  # Fome continua aumentando

    def see_food(self, food_x, food_y):
        """Verifica se a comida está no raio de visão"""
        if not self.alive:
            return False
        distance = math.sqrt((self.x - food_x) ** 2 + (self.y - food_y) ** 2)
        return distance <= self.vision

    def die_if_hungry(self):
        """Mata o agente se ele estiver faminto demais"""
        if self.hunger >= 100:
            self.alive = False


# Classe da Simulação
class Simulation:
    def __init__(self, grid_size, agents, hunger):
        self.grid_size = grid_size
        self.food_x = random.randint(0, grid_size - 1)
        self.food_y = random.randint(0, grid_size - 1)
        self.agents = [
            Agent(random.randint(0, grid_size - 1), random.randint(0, grid_size - 1), hunger)
            for _ in range(agents)
        ]
        self.turn = 0  # Turno atual

    def step(self):
        """Executa um turno da simulação"""
        self.turn += 1
        for agent in self.agents:
            if not agent.alive:
                continue
            if agent.see_food(self.food_x, self.food_y):
                agent.move_towards(self.food_x, self.food_y)
                if agent.x == self.food_x and agent.y == self.food_y:
                    print(f"Agente comeu a comida em ({self.food_x}, {self.food_y})!")
                    agent.hunger = 0  # Fome é zerada
                    self.food_x = random.randint(0, self.grid_size - 1)
                    self.food_y = random.randint(0, self.grid_size - 1)
            else:
                agent.move(self.grid_size)
            agent.die_if_hungry()

    def display(self):
        """Mostra o estado atual do mapa"""
        print(f"\nTurno: {self.turn}")
        print("Mapa do Jogo:")
        for y in range(self.grid_size):
            row = ""
            for x in range(self.grid_size):
                if any(agent.x == x and agent.y == y and agent.alive for agent in self.agents):
                    row += "A "  # Agente
                elif self.food_x == x and self.food_y == y:
                    row += "F "  # Comida
                else:
                    row += ". "
            print(row)
        print("\nStatus dos Agentes:")
        for idx, agent in enumerate(self.agents):
            status = "Vivo" if agent.alive else "Morto"
            print(f"Agente {idx + 1}: Posição ({agent.x}, {agent.y}), Fome: {agent.hunger}, Status: {status}")
        print("-" * 40)


# Configuração da simulação
grid_size = 10  # Tamanho do mapa
initial_hunger = 0  # Fome inicial dos agentes
num_agents = 1  # Número de agentes

# Inicializa a simulação
sim = Simulation(grid_size, num_agents, initial_hunger)


In [26]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Criação do controle com botões de incremento/decremento
turn_selector = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100,
    step=1,
    description="Turno:",
    style={'description_width': 'initial'}
)

def update_simulation(turn):
    """Atualiza a simulação até o turno atual"""
    while sim.turn < turn:  # Avança até o turno selecionado
        sim.step()
    clear_output(wait=True)
    sim.display()

# Conectar o controle ao valor
interactive_widget = widgets.interactive(update_simulation, turn=turn_selector)

# Mostrar o controle com botões
display(interactive_widget)


interactive(children=(BoundedIntText(value=0, description='Turno:', style=DescriptionStyle(description_width='…